In [1]:
pip install pygame

Note: you may need to restart the kernel to use updated packages.Requirement already satisfied: pygame in d:\setups\anaconda\lib\site-packages (2.6.1)



In [2]:
pip install torch torchvision torchaudio

In [3]:
pip install typing_extensions --upgrade


Note: you may need to restart the kernel to use updated packages.


In [5]:
import pygame
import math
import torch
import torch.nn as nn
import torch.optim as optim
from random import randint as ri

pygame.init()

# GAME Parameters
screen = pygame.display.set_mode([900, 800])
GAME_x = 40
GAME_y = 60
GAME_width = 780
GAME_height = 560
GAME_border = 3
WHITE = (255, 255, 255)
BLUE = (0, 0, 255)
BLACK = (0, 0, 0)
RED = (255, 0, 0)
GREEN = (0, 255, 0)
GRAY = (150, 150, 150)
custom_color_1 = (10, 145, 80)

# Game modes
MODE_OBSTACLE = 'obstacle'
MODE_START = 'start'
MODE_GOAL = 'goal'
MODE_ERASER = 'eraser'
MODE_RESET = 'reset'
MODE_PLAY = 'play'

# Game state
current_mode = MODE_OBSTACLE
start_point = None
end_point = None
Step = 10
parent = {}
Trace = None
obstacles = set()

# Neural Network Model for RRT Guidance
class NeuralRRTModel(nn.Module):
    def __init__(self):
        super(NeuralRRTModel, self).__init__()
        self.fc1 = nn.Linear(4, 64)
        self.fc2 = nn.Linear(64, 64)
        self.fc3 = nn.Linear(64, 2)

    def forward(self, start_goal):
        x = torch.relu(self.fc1(start_goal))
        x = torch.relu(self.fc2(x))
        return self.fc3(x)

# Initialize neural network and optimizer
model = NeuralRRTModel()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Class Definition for Button
class Button:
    def __init__(self, colour, x, y, width, height):
        self.colour = colour
        self.x = x
        self.y = y
        self.width = width
        self.height = height

    def create(self, screen):
        pygame.draw.rect(screen, self.colour, [self.x, self.y, self.width, self.height])

def reset_game():
    global start_point, end_point, obstacles, parent, Trace
    screen.fill(WHITE)
    pygame.draw.rect(screen, BLACK, (GAME_x, GAME_y, GAME_width, GAME_height), GAME_border)
    start_point = None
    end_point = None
    parent = {}
    Trace = None
    obstacles.clear()

def is_colliding(x, y):
    if not point_inside_game(x, y) or screen.get_at((x, y)) == BLACK:
        return True
    return False

def is_collision_free(x1, y1, x2, y2):
    dx = x2 - x1
    dy = y2 - y1
    distance = int(((dx)**2 + (dy)**2) ** 0.5)

    for step in range(distance):
        t = step / distance
        x_check = int(x1 + t * dx)
        y_check = int(y1 + t * dy)
        if screen.get_at((x_check, y_check)) == BLACK:
            return False
    return True

def random_point():
    return (ri(GAME_x + GAME_border, GAME_x + GAME_width - GAME_border),
            ri(GAME_y + GAME_border, GAME_y + GAME_height - GAME_border))

def p2p_dist(p1, p2):
    return math.sqrt((p1[0] - p2[0])**2 + (p2[1] - p1[1])**2)

def get_new_point(x_m, y_m, x, y):
    angle = math.atan2(y - y_m, x - x_m)
    x_new = x_m + Step * math.cos(angle)
    y_new = y_m + Step * math.sin(angle)
    return int(x_new), int(y_new)

# Neural-guided RRT (Neural Network assisted sampling)
def neural_guided_sampling(nn_model, start, goal):
    input_tensor = torch.tensor([start[0], start[1], goal[0], goal[1]], dtype=torch.float32)
    output = nn_model(input_tensor).detach().numpy()
    return int(output[0]), int(output[1])

# RRT Function 
def RRT(x, y, parent):
    min_dist = float('inf')
    nearest_point = None

    for point in parent:
        dist = p2p_dist(point, (x, y))
        if dist < min_dist:
            min_dist = dist
            nearest_point = point

    if nearest_point:
        x_m, y_m = nearest_point
        x_new, y_new = get_new_point(x_m, y_m, x, y)

        if is_collision_free(x_m, y_m, x_new, y_new):
            return True, x_m, y_m, (x_new, y_new)

    return False, None, None, None

def rrt_algorithm():
    global Trace, start_point, end_point
    parent[start_point] = (-1, -1)
    Trace = []
    fail_counter = 0
    rrt_running = True

    while rrt_running:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                return

        if fail_counter > 20: 
            break

        # Neural-guided sampling
        x, y = neural_guided_sampling(model, start_point, end_point)

        if is_colliding(x, y):
            x, y = random_point()

        success, x_m, y_m, new_point = RRT(x, y, parent)

        if success:
            parent[new_point] = (x_m, y_m)
            pygame.draw.line(screen, BLUE, (x_m, y_m), new_point, 2)
            Trace.append(new_point)
            pygame.display.update()  
            fail_counter = 0  

            if p2p_dist(new_point, end_point) < Step:
                rrt_running = False
                trace_optimal_path(new_point)
                return True
        else:
            fail_counter += 1  

        pygame.display.update()

    return False

# Initialize neural network and optimizer
model = NeuralRRTModel()
optimizer = optim.Adam(model.parameters(), lr=0.001)


# Interface buttons
buttons = {
    'obstacle': Button(BLACK, 50, 700, 120, 60),
    'start': Button(RED, 190, 700, 120, 60),
    'goal': Button(GREEN, 330, 700, 120, 60),
    'eraser': Button(GRAY, 470, 700, 120, 60),
    'reset': Button(BLUE, 610, 700, 120, 60),
    'play': Button(custom_color_1, 750, 700, 120, 60)
}

def point_inside_game(x, y):
    return GAME_x + GAME_border < x < GAME_x + GAME_width - GAME_border and \
           GAME_y + GAME_border < y < GAME_y + GAME_height - GAME_border

def point_inside_rec(xr, yr, wr, hr, x, y):
    return xr < x < xr + wr and yr < y < hr + yr

def draw_buttons():
    for button in buttons.values():
        button.create(screen)
    font = pygame.font.Font('freesansbold.ttf', 16)
    screen.blit(font.render('Obstacle', True, WHITE), (65, 725))
    screen.blit(font.render('Start', True, WHITE), (225, 725))
    screen.blit(font.render('Goal', True, WHITE), (370, 725))
    screen.blit(font.render('Eraser', True, WHITE), (505, 725))
    screen.blit(font.render('Reset', True, WHITE), (645, 725))
    screen.blit(font.render('Play', True, WHITE), (785, 725))

def DesText(s, x=400, y=640):
    pygame.draw.rect(screen, WHITE, (250, 620, 400, 20))  
    font = pygame.font.SysFont('segoeuisemilight', 20)
    text = font.render(f"{s}", True, BLACK)  
    screen.blit(text, (x, y))

def handle_button_click(x, y):
    global current_mode
    if point_inside_rec(buttons['obstacle'].x, buttons['obstacle'].y, buttons['obstacle'].width, buttons['obstacle'].height, x, y):
        current_mode = MODE_OBSTACLE
    elif point_inside_rec(buttons['start'].x, buttons['start'].y, buttons['start'].width, buttons['start'].height, x, y):
        current_mode = MODE_START
    elif point_inside_rec(buttons['goal'].x, buttons['goal'].y, buttons['goal'].width, buttons['goal'].height, x, y):
        current_mode = MODE_GOAL
    elif point_inside_rec(buttons['eraser'].x, buttons['eraser'].y, buttons['eraser'].width, buttons['eraser'].height, x, y):
        current_mode = MODE_ERASER
    elif point_inside_rec(buttons['reset'].x, buttons['reset'].y, buttons['reset'].width, buttons['reset'].height, x, y):
        current_mode = MODE_RESET
    elif point_inside_rec(buttons['play'].x, buttons['play'].y, buttons['play'].width, buttons['play'].height, x, y):
        current_mode = MODE_PLAY


# Function to trace back and highlight the optimal path in green
def trace_optimal_path(current_point):
    global parent
    while current_point != start_point:
        prev_point = parent[current_point]
        pygame.draw.line(screen, GREEN, current_point, prev_point, 3)  # Draw path in green
        current_point = prev_point
        pygame.display.update()
    
    # After tracing the path, freeze the screen for user inspection
    freeze_screen()

# Function to freeze the screen until user decides to continue
def freeze_screen():
    frozen = True
    while frozen:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                return
            elif event.type == pygame.KEYDOWN or event.type == pygame.MOUSEBUTTONDOWN:
                frozen = False  # Break the loop if any key or mouse click occurs

# Main Game Loop
running = True
reset_game()
dragging = False
while running:
    screen.fill(WHITE)
    pygame.draw.rect(screen, BLACK, (GAME_x, GAME_y, GAME_width, GAME_height), GAME_border)
    draw_buttons()
    DesText(f"Current Mode: {current_mode}")

    # Draw persistent obstacles, start, and end points
    for obstacle in obstacles:
        pygame.draw.circle(screen, BLACK, obstacle, 10)
    if start_point:
        pygame.draw.circle(screen, RED, start_point, 10)
    if end_point:
        pygame.draw.circle(screen, GREEN, end_point, 10)

    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
        elif event.type == pygame.MOUSEBUTTONDOWN:
            x, y = event.pos
            handle_button_click(x, y)

            if current_mode == MODE_OBSTACLE and point_inside_game(x, y):
                pygame.draw.circle(screen, BLACK, event.pos, 10)
                obstacles.add(event.pos)
                dragging = True

            elif current_mode == MODE_START and point_inside_game(x, y):
                start_point = event.pos
                pygame.draw.circle(screen, RED, start_point, 10)

            elif current_mode == MODE_GOAL and point_inside_game(x, y):
                end_point = event.pos
                pygame.draw.circle(screen, GREEN, end_point, 10)

            elif current_mode == MODE_ERASER and point_inside_game(x, y):
                for obstacle in list(obstacles):  # Copy set to list to avoid modifying while iterating
                    if p2p_dist(event.pos, obstacle) < 10:  # Check if the mouse is close to an obstacle
                        obstacles.remove(obstacle)
                        pygame.draw.circle(screen, WHITE, obstacle, 10)  # Erase obstacle
                dragging = True

            elif current_mode == MODE_RESET:
                reset_game()

            elif current_mode == MODE_PLAY and start_point and end_point:
                rrt_algorithm()

        elif event.type == pygame.MOUSEBUTTONUP:
            dragging = False

        elif event.type == pygame.MOUSEMOTION:
            if dragging and current_mode == MODE_OBSTACLE:
                x, y = event.pos
                if point_inside_game(x, y):
                    pygame.draw.circle(screen, BLACK, event.pos, 10)
                    obstacles.add(event.pos)
            elif dragging and current_mode == MODE_ERASER:
                x, y = event.pos
                for obstacle in list(obstacles):  # Copy set to list to avoid modifying while iterating
                    if p2p_dist(event.pos, obstacle) < 10:  # Check if the mouse is close to an obstacle
                        obstacles.remove(obstacle)
                        pygame.draw.circle(screen, WHITE, obstacle, 10)  # Erase obstacle

    pygame.display.update()

pygame.quit() 


error: video system not initialized